In [1]:
import os
import glob
from datetime import datetime
from multiprocessing import Pool

import matplotlib.pyplot as plt

from uncertainty import ComputeTotUncertainty

%matplotlib inline

In [3]:
sid_dir = 'data2/Anton/sia_sid_cdr_postproc'
sia_dir = 'data2/Anton/sia/cdr_1991_2023'
mesh_dir = f'{sia_dir}/mesh'
unc_dir = f'{sia_dir}/unc'
age_dir = f'{sia_dir}/age'


mesh_init_file = 'mesh_arctic_ease_25km_max7.npz'

ifiles = sorted(glob.glob(f'{sid_dir}/*/ice_drift*npz'))
idates = [datetime.strptime(os.path.basename(ifile).split('-')[-1].split('.')[0], '%Y%m%d%H%M')
          for ifile in ifiles]

start_index = idates.index(datetime(1991, 9, 15, 12))

In [ ]:
idate = datetime(2024, 12, 31)
#idate = datetime(2020,12,31)

comp = ComputeTotUncertainty(mesh_dir, unc_dir, sid_dir, age_dir, mesh_init_file)
a, c, f, x, y, t = comp.load_age(idate)

print(f.shape)
plt.tripcolor(x, y, t, f[0])
plt.show()

unc_obs = comp.load_unc_obs(idate)
plt.tripcolor(x, y, t, unc_obs)
plt.colorbar()
plt.show()

unc_myi = comp.load_unc_myi(idate)
plt.tripcolor(x, y, t, unc_myi[0]) 
plt.colorbar()
plt.show()

unc_sid = comp.load_unc_sid(idate, unc_obs)
plt.tripcolor(x, y, t, unc_sid[1])
plt.colorbar()
plt.show()

unc_sic_sid = comp.load_unc_sidsic(idate, unc_obs)
plt.tripcolor(x, y, t, unc_sic_sid[1])
plt.colorbar()
plt.show()

unc_frac = comp.compute_unc_frac(unc_obs, unc_myi)
plt.tripcolor(x, y, t, unc_frac[0])
plt.colorbar()
plt.show()

unc_tot, unc_age = comp.compute_unc_tot(unc_frac, unc_sic_sid)

plt.tripcolor(x, y, t, unc_tot[1])
plt.colorbar()
plt.show()

plt.tripcolor(x, y, t, unc_age)
plt.colorbar()
plt.show()

In [ ]:
fig, axs = plt.subplots(len(f), 5, figsize=(14, 20), sharex=True, sharey=True)
for i in range(len(f)):
    trp0 = axs[i, 0].tripcolor(x, y, t, f[::-1][i], clim=[0, 100], cmap='jet')
    trp1 = axs[i, 1].tripcolor(x, y, t, unc_frac[i], clim=[0, 10], cmap='jet')
    trp2 = axs[i, 2].tripcolor(x, y, t, unc_sid[i], clim=[0, 300], cmap='jet')    
    trp3 = axs[i, 3].tripcolor(x, y, t, unc_sic_sid[i], clim=[0, 25], cmap='jet')
    trp4 = axs[i, 4].tripcolor(x, y, t, unc_tot[i], clim=[0, 25], cmap='jet')    
    
for ax in axs.flat:
    ax.set_aspect('equal')
    ax.set_axis_off()

axs[0, 0].set_title('SIA fraction, 0 - 100 %')
axs[0, 1].set_title('SIC Uncertainty 0 - 10 %')
axs[0, 2].set_title('SID Uncertainty, 0 - 300 km')
axs[0, 3].set_title('SIC<-SID Uncertainty, 0 - 25 %')
axs[0, 4].set_title('Total Uncertainty, 0 - 25 %')

plt.tight_layout()
plt.show()

In [ ]:
comp = ComputeTotUncertainty(mesh_dir, unc_dir, sid_dir, age_dir, mesh_init_file)
with Pool(6) as pool:
    pool.map(comp.compute_uncertainty, idates[start_index:])